# Objective: Next word prediction using RNN - LSTM

##  If given the three words of a sentence, it should be able to guess the fourth.

![](Image.jpg)

In [1]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation

# 1.Import libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import pickle
import numpy as np
import os

# 2. Open and pre-process the data

In [3]:
file = open(r"Pride and Prejudice.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
    data = ' '. join(lines)

# replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Pride and Prejudice This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook. Title:'

# 3. Implement tokenization and make additional adjustments

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)


131180

In [5]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7252


In [6]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  131177


array([[   1,  181,  390, 1000],
       [ 181,  390, 1000,    3],
       [ 390, 1000,    3,  298],
       [1000,    3,  298,    4],
       [   3,  298,    4,  946],
       [ 298,    4,  946,   41],
       [   4,  946,   41, 1000],
       [ 946,   41, 1000,   23],
       [  41, 1000,   23,   21],
       [1000,   23,   21,    1]])

In [7]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)

In [8]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 4. Creating the model

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(500, return_sequences=True))
model.add(LSTM(500))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 10)             72520     
                                                                 
 lstm_2 (LSTM)               (None, 3, 500)            1022000   
                                                                 
 lstm_3 (LSTM)               (None, 500)               2002000   
                                                                 
 dense_2 (Dense)             (None, 1000)              501000    
                                                                 
 dense_3 (Dense)             (None, 7252)              7259252   
                                                                 
Total params: 10856772 (41.42 MB)
Trainable params: 10856772 (41.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 5. Train the model

In [11]:

model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
model.fit(X, y, epochs=30, batch_size=64)


Epoch 1/30
2050/2050 [==============================] - 36s 14ms/step - loss: 6.2696 - accuracy: 0.0554
Epoch 2/30
2050/2050 [==============================] - 24s 12ms/step - loss: 5.6521 - accuracy: 0.1021
Epoch 3/30
2050/2050 [==============================] - 24s 12ms/step - loss: 5.2993 - accuracy: 0.1240
Epoch 4/30
2050/2050 [==============================] - 24s 12ms/step - loss: 5.0591 - accuracy: 0.1374
Epoch 5/30
2050/2050 [==============================] - 24s 12ms/step - loss: 4.8418 - accuracy: 0.1489
Epoch 6/30
2050/2050 [==============================] - 23s 11ms/step - loss: 4.6329 - accuracy: 0.1580
Epoch 7/30
2050/2050 [==============================] - 23s 11ms/step - loss: 4.4252 - accuracy: 0.1670
Epoch 8/30
2050/2050 [==============================] - 24s 12ms/step - loss: 4.2162 - accuracy: 0.1760
Epoch 9/30
2050/2050 [==============================] - 24s 12ms/step - loss: 4.0034 - accuracy: 0.1874
Epoch 10/30
2050/2050 [==============================] - 24s 12m

In [12]:
model.save("next_word_model.h5")
model.save("next_word_model.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 6. Let’s predict

In [13]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_word_model.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""

    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break

    print(predicted_word)
    return predicted_word

In [14]:
while(True):
    text = input("Enter your line: ")

    if text == "0":
        print("Execution completed.....")
        break

    else:
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)

            Predict_Next_Words(model, tokenizer, text)

        except Exception as e:
            print("Error occurred: ",e)
            continue

Enter your line: ebook is for the use of anyone
['use', 'of', 'anyone']
1/1 [==============================] - 1s 1s/step
anywhere
Enter your line: If you are not
['you', 'are', 'not']
1/1 [==============================] - 0s 18ms/step
located
Enter your line: you will have
['you', 'will', 'have']
1/1 [==============================] - 0s 19ms/step
the
Enter your line: country where you
['country', 'where', 'you']
1/1 [==============================] - 0s 19ms/step
are
Enter your line: 0
Execution completed.....
